# Requirements

In [1]:
import altair as alt
import datetime
import pandas as pd
import panel as pn
import re

css = '''
table {
    border-collapse: collapse;
}
table, th, td {
    border: 1px solid black;
}
th, td {
    padding: 3px;
}
'''

pn.extension('vega', raw_css=[css])

# Import data

In [2]:
url = 'Schooltool-latest.xlsx'
df = pd.read_excel(url, sheet_name='Data')

# Preprocessing

## Sanitize data

In [3]:
df.rename(columns={
    'Gemeente': 'Naam',
    'Bevol-king 2020': 'Bevolking',
}, inplace=True)

In [4]:
df.drop(columns=['Niscode'], inplace=True)

In [5]:
column_names = list()
for column in df.columns:
    if type(column) != str:
        column_names.append(datetime.datetime.strftime(column, '%Y-%m-%d'))
    else:
        match = re.match(r'(\d{4}-\d{2}-\d{2}).*', column)
        if match:
            column_names.append(match.group(1))
        else:
            column_names.append(column)
df.columns = column_names

## Extract data

Extract the data for municipalities, regions, and arrondissements into their own dataframes.

In [6]:
ranges = {
    'gemeenten': slice(0, 589),
    'regios': slice(593, 607),
    'Belgie': slice(592, 593),
}

In [9]:
df_cases = dict()
for category in ranges:
    df_cases[category] = df.iloc[ranges[category], :].copy()

Take out metadata for each category.

In [10]:
df_metas = dict()
for category, df_case in df_cases.items():
    df_metas[category] = df_case[['Naam', 'Bevolking']]
    df_case.drop(columns='Bevolking', inplace=True)

In [15]:
def clean_names(df):
    df.loc[:, 'Naam'] = df['Naam'].apply(lambda x: x.strip().strip("'"))

In [16]:
for category in df_cases:
    clean_names(df_cases[category])
    clean_names(df_metas[category])

# Derived data functions

Function to select a particular city or region from the original dataframe with case data.

In [17]:
def select_name(df, name):
    return df.query(f'Naam == "{name}"').melt('Naam', var_name='Datum', value_name='Dagcijfers')

In [18]:
def get_population(df, name):
    try:
        return df.query(f'Naam == "{name}"').Bevolking.values[0]
    except:
        raise IndexError(f'no value for {name}')

In [19]:
def add_plot_values(df_orig, bevolking):
    df = df_orig.copy()
    df['7-daags gemiddelde'] = df.Dagcijfers.rolling(7).mean()
    df['2 weken cijfers'] = df.Dagcijfers.rolling(14).sum()
    df['twee_weken_incidentie'] = 100_000*df['2 weken cijfers']/bevolking
    return df

In [20]:
def add_text_values(df_orig, bevolking):
    df = df_orig.copy()
    df['Week gemiddelde'] = df.Dagcijfers.rolling(7).mean()
    df['2 weken gemiddelde'] = df.Dagcijfers.rolling(14).mean()
    df['Weekcijfers'] = df.Dagcijfers.rolling(7).sum()
    df['2 weken cijfers'] = df.Dagcijfers.rolling(14).sum()
    df['Dag incidentie'] = 100_000*df.Dagcijfers/bevolking
    df['Week incidentie'] = 100_000*df.Weekcijfers/bevolking
    df['2 weken incidentie'] = 100_000*df['2 weken cijfers']/bevolking
    return df

In [21]:
def get_text_values(df_orig, bevolking):
    df = add_text_values(df_orig, bevolking)
    last_date = df.Datum.max()
    data = df.query(f'Datum == "{last_date}"').to_dict(orient='list')
    return {k:v[0] for k, v in data.items()}

# Dashboard

In [22]:
def round_up(value):
    return int(round(value, 0))

## Plot

In [23]:
def create_plot(df_all, df_meta_all, name, extra_thresholds=None):
    population = get_population(df_meta_all, name)
    df = add_plot_values(select_name(df_all, name), population)
    df['Dagcijfers_tooltip'] = df.Dagcijfers.apply(round_up)
    
    base = alt.Chart(df).encode(
        alt.X('Datum:T', axis=alt.Axis(title=None))
    )
    dagcijfers = base.mark_bar(opacity=0.6).encode(
        alt.Y('Dagcijfers:Q', axis=alt.Axis(title='Bijkomende besmettingen')),
        tooltip=['Datum:T', 'Dagcijfers_tooltip:Q'],
    )
    
    dagcijfers_y_dagen = base.mark_line(strokeDash=(5, 2), opacity=0.6).encode(
        alt.Y('7-daags gemiddelde:Q', axis=alt.Axis(title='')),
    )

    incidentie_2_weken = base.mark_line(stroke='green').encode(
        alt.Y('twee_weken_incidentie:Q', axis=alt.Axis(title='2 weken incidentie')),
    )
    max_2_weken_incidentie = df.twee_weken_incidentie.max()
    
    incident_plots = incidentie_2_weken
    if extra_thresholds:
        for threshold in extra_thresholds:
            if threshold > max_2_weken_incidentie:
                break
            df_threshold = pd.DataFrame({
                'Datum': [df.Datum.min(), df.Datum.max()],
                'Drempel': [threshold, threshold],
            })
            threshold_plot = alt.Chart(df_threshold).mark_line(stroke='red', size=0.8).encode(
                alt.X('Datum:T'),
                alt.Y('Drempel:Q', axis=alt.Axis(title='')),
            )
            incident_plots += threshold_plot

    return alt.layer(dagcijfers + dagcijfers_y_dagen, incident_plots).resolve_scale(
        y='independent'
    ).properties(
        width=700,
        height=350
    )

## Table

In [24]:
def create_text(df_all, df_meta_all, name):
    df = select_name(df_all, name)
    bevolking = get_population(df_meta_all, name)
    values = get_text_values(df, bevolking)
    datum = values['Datum']
    dag_aantal = round_up(values['Dagcijfers'])
    dag_incidentie = round_up(values['Dag incidentie'])
    week_aantal = round_up(values['Weekcijfers'])
    week_gemiddelde = round_up(values['Week gemiddelde'])
    week_incidentie = round_up(values['Week incidentie'])
    week_2_aantal = round_up(values['2 weken cijfers'])
    week_2_gemiddelde = round_up(values['2 weken gemiddelde'])
    week_2_incidentie = round_up(values['2 weken incidentie'])
    text = ''
    text += f'| {name}                      | Periode | Aantal          | Incidentie          |\n'
    text += f'|-----------------------------|---------|-----------------|---------------------|\n'
    text += f'| Bevolking: {int(bevolking)} | Dag     | {dag_aantal}    | {dag_incidentie}    |\n'
    text += f'| Datum: {datum}              | Week    | {week_aantal}   | {week_incidentie}   |\n'
    text += f'|                             | 2 weken | {week_2_aantal} | {week_2_incidentie} |\n'
    return text

## Panel

In [25]:
extra_thresholds = [50.0, 100.0]

In [27]:
gemeente = pn.widgets.Select(options=sorted(list(df_metas['gemeenten'].Naam)), name='Gemeente')
regio = pn.widgets.Select(options=list(df_metas['regios'].Naam), name='Regio')

@pn.depends(gemeente.param.value)
def get_municipality_plot(gemeente):
    return create_plot(df_cases['gemeenten'], df_metas['gemeenten'], gemeente,
                       extra_thresholds=extra_thresholds)

@pn.depends(regio.param.value)
def get_regional_plot(regio):
    return create_plot(df_cases['regios'], df_metas['regios'], regio,
                       extra_thresholds=extra_thresholds)

@pn.depends(gemeente.param.value)
def get_municipality_text(gemeente):
    return create_text(df_cases['gemeenten'], df_metas['gemeenten'], gemeente)

@pn.depends(regio.param.value)
def get_regional_text(regio):
    return create_text(df_cases['regios'], df_metas['regios'], regio)

header_style = {
    'background': "#D64B38",
    'width':"100%",
    'color':'white',
    'padding':'10px'
}
plot_margin = (10, 200, 10, 10)
text_margin = (10, 10, 10, 200)

municipality_tab = pn.Column(
    pn.Row(gemeente),
    pn.Row(
        pn.Column(get_municipality_plot, margin=plot_margin),
        pn.Column(get_municipality_text, margin=text_margin),
    ),
)
regional_tab = pn.Column(
    pn.Row(regio),
    pn.Row(
        pn.Column(get_regional_plot, margin=plot_margin),
        pn.Column(get_regional_text, margin=text_margin),
    ),
)
belgie_tab = pn.Column(
    pn.Row(
        pn.Column(create_plot(df_cases['Belgie'], df_metas['Belgie'], 'BELGIË',
                              extra_thresholds=extra_thresholds),
                  margin=plot_margin),
        pn.Column(create_text(df_cases['Belgie'], df_metas['Belgie'], 'BELGIË'),
                  margin=text_margin)
    )
)
over_str = \
'''
## Data

Data betrokken van [Sciensano publieke datasets](https://epistat.wiv-isp.be/covid/), herberekend
door Jan Hertogen

## Dashboard

Opgemaakt door (in alfabetisch volgorde): Geert Jan Bex, Jan Hertogen, Geert Molenberghs
'''

over_tab = pn.Column(
    pn.pane.Markdown(over_str),
)

pn.Column(
    pn.pane.Markdown('# School tool', styles=header_style),
    pn.pane.Markdown('### Besmettingen (staven) en 7-daags gemiddelde (stippenlijn) op linkeras, 2 weken incidentie (groene lijn) en 50-lijn op rechteras.'),
    pn.Tabs(
        ('Gemeenten', municipality_tab),
        ('Regio\'s', regional_tab),
        ('Belgie', belgie_tab),
        ('Over...', over_tab),
    )
).servable()

/home/gjb/mambaforge/envs/python_dashboards/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/gjb/mambaforge/envs/python_dashboards/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/gjb/mambaforge/envs/python_dashboards/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if

Column
    [0] Markdown(str, styles={'background': '#D64B38', ...})
    [1] Markdown(str)
    [2] Tabs
        [0] Column
            [0] Row
                [0] Select(name='Gemeente', options=['AALST', 'AALTER', ...], value='AALST')
            [1] Row
                [0] Column(margin=(10, 200, 10, 10))
                    [0] ParamFunction(function, _pane=Vega, defer_load=False)
                [1] Column(margin=(10, 10, 10, 200))
                    [0] ParamFunction(function, _pane=Markdown, defer_load=False)
        [1] Column
            [0] Row
                [0] Select(name='Regio', options=['VLAAMS GEWEST', ...], value='VLAAMS GEWEST')
            [1] Row
                [0] Column(margin=(10, 200, 10, 10))
                    [0] ParamFunction(function, _pane=Vega, defer_load=False)
                [1] Column(margin=(10, 10, 10, 200))
                    [0] ParamFunction(function, _pane=Markdown, defer_load=False)
        [2] Column
            [0] Row
                [0] Column(margin=(10, 200, 10, 10))
                    [0] Vega(LayerChart, selection=Selection)
                [1] Column(margin=(10, 10, 10, 200))
                    [0] Markdown(str)
        [3] Column
            [0] Markdown(str)